### __Option Pricing on S&P 500 Daily Risk Control 10% Index__

__The S&P 500 Daily Risk Control 10% Index (SPXT10UE) is part of S&P Dow Jones Risk Control Indices familly and use the below parameters__

| ***Index Name*** | ***Underlying Risk Index*** | ***Risk Control Level*** | ***Maximum Leverage*** | ***Interest Rate*** | ***Volatility Calculation*** | ***Return Frequency for Volatility*** | ***Lag to Rebalancing Date*** | ***Decay Factor (Short-Term)*** | ***Decay Factor (Long-Term)*** | ***Rebalancing Frequency*** | ***Launch Date*** | ***Bloomberg Tickers***                                                           |
|------------------------------------------------|------------------------------------------------------|------------------------|----------------------|-------------------------|----------------------------|--------------------------------------|-----------------------------|-------------------------------|------------------------------|---------------------------|---------------|---------------------------------------------------------------------------------|
| S&P 500 Daily Risk Control 10% Index|S&P 500 Total Return: SPTR (USD) | 10% | 150%                 | SOFR + 0.02963*         | Exponentially weighted    | Daily                                | 2 days                      | 94%                           | 97%                          | Daily                     | 10-Sep-09     | ***Excess Return:*** SPXT10UE (USD)|


__The S&P 500 Daily Risk Control indices are computed using the below methodology__

$$
\text{Risk Control ER Index Value}_t 
= 
\text{RiskControlERIndexValue}_{rb}
\,\times\,
\Biggl[
1
\;+\;
K_{rb}\,\biggl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-1}} \;-\; 1\biggr)
\;-\;
K_{rb}\,\Bigl(
  \prod_{i=rb+1}^{t}\bigl(1 + \text{InterestRate}_{i-1} \times \frac{D_{i-1,i}}{360}\bigr)
  \;-\; 1
\Bigr)
\Biggr]
$$


$$
K_{rb} 
= 
\min\!\Bigl(\text{Max }K,\;\frac{\text{Target Volatility}}{\text{Realized Volatility}_{rb-d}}\Bigr)
$$


$$
\text{RealizedVolatility}_t 
  = \max\bigl(\text{RealizedVolatility}_{S,t}, \text{RealizedVolatility}_{L,t}\bigr)
$$

$$
\text{RealizedVolatility}_{S,t} 
  = \sqrt{\frac{252}{n}\,\text{Variance}_{S,t}}
$$

$$
\text{RealizedVolatility}_{L,t}
  = \sqrt{\frac{252}{n}\,\text{Variance}_{L,t}}
$$

$$
\text{Variance}_{S,t}
  = \lambda_S\,\text{Variance}_{S,t-1}
   + \bigl(1 - \lambda_S\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

$$
\text{Variance}_{L,t}
  = \lambda_L\,\text{Variance}_{L,t-1}
   + \bigl(1 - \lambda_L\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

__Excess Return Volatility Target SDE with SABR Model__


\begin{aligned}
\frac{dV_t}{V_t} &= K_{rb}\Bigl(g \cdot dt + \frac{dF_t}{F_t}\Bigr)
\end{aligned}

\begin{aligned}
\frac{dF_t}{F_t}&= \alpha_t dW_t
\end{aligned}

\begin{aligned}
d\alpha_t &= \nu\, \alpha_t\, dZ_t
\end{aligned}

\begin{aligned}
\mathrm{corr}(dW_t,dZ_t) &= \rho
\end{aligned}

\begin{aligned}
\frac{dS_t}{S_t}&= (r+g)\cdot dt + \frac{dF_t}{F_t}
\end{aligned}

In [1]:
# -------
# IMPORT
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import BlackScholesModel, SABRModel

In [88]:
# -------
# INPUTS
# -------
F = 100.0
T = 1.0
alpha = 0.1 
beta = 1.0 
rho = -0.9
nu = 0.9
n_steps = 250
n_paths = 2
seed = True
r = 0.03
g = 0.0020
dt = T / n_steps
vol_target = 0.10
max_leverage = 1.5

In [130]:
# -------
# Simulate 1Y of risky asset price paths
# -------
index_fwd_process, _ = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, n_steps=n_steps, n_paths=1, seed=seed, seed_value=1)
index_returns = (index_fwd_process.pct_change() + r*dt + g*dt).fillna(0)
scaling = F / index_returns.add(1).cumprod().iloc[-1].iloc[0]

In [139]:
# -------
# Simulate path for pricing
# -------
fwd_process, _ = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, n_steps=n_steps, n_paths=n_paths, seed=seed, seed_value=47)
eq_return_process = (fwd_process.pct_change() + r*dt + g*dt).fillna(0)

# Create a DataFrame by repeating index_returns values for each column in eq_return_process
index_returns_df = pd.DataFrame(
    data=np.tile(index_returns.values, (1, len(eq_return_process.columns))), 
    index=index_returns.index, 
    columns=eq_return_process.columns
)

eq_return_process = pd.concat([index_returns_df, eq_return_process], axis=0, ignore_index=True)
eq_return_process = eq_return_process.add(1).cumprod()  * scaling

log_returns_squared = np.log(eq_return_process / eq_return_process.shift(1)).pow(2)

# Short-term variance
variance_S = log_returns_squared.ewm(alpha=0.5, adjust=False).mean()
variance_S = np.sqrt(variance_S * 250)

# Long-term variance
variance_L = log_returns_squared.ewm(alpha=0.01, adjust=False).mean()
variance_L = np.sqrt(variance_L * 250)

realized_variance = variance_S.combine(variance_L, np.maximum)

k = (vol_target / realized_variance).clip(upper=max_leverage).fillna(1.0)
k = k.shift(2).fillna(1.0)

In [144]:
px.line(variance_S)

In [140]:
realized_variance

,0,1
0,NaN,NaN
1,0.163609,0.163609
2,0.162877,0.162877
3,0.162130,0.162130
4,0.161603,0.161603
...,...,...
497,0.075096,0.070206
498,0.074741,0.075477
499,0.074626,0.069973
500,0.076683,0.069637


In [141]:
k = k.loc[index_returns_df.index[-1]:].reset_index(drop=True)

In [135]:
fwd_process.pct_change()

,0,1
0,NaN,NaN
1,0.003141,-0.007975
2,-0.000857,0.006244
3,0.003945,0.013442
4,0.008875,0.005996
...,...,...
246,-0.002046,0.001535
247,-0.002986,-0.002909
248,-0.004257,-0.000567
249,0.011853,-0.002496


In [142]:
k

,0,1
0,1.500000,1.500000
1,1.500000,1.500000
2,1.500000,1.500000
3,1.500000,1.500000
4,1.500000,1.188138
...,...,...
247,1.325204,1.424769
248,1.331222,1.429609
249,1.331631,1.424390
250,1.337957,1.324911
